In [1]:
#%autosave 0

In [2]:
!pip install transformers

In [3]:
!pip install ipywidgets widgetsnbextension pandas-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 8.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 47.4 MB/s eta 0:00:00a 0:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.8/357.8 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 81.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 85.2 MB/s eta 0:00:00ta 0:00:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=b152a40c0a741c6cf0146deccce6b5783742b5a2f4725c68262847bd0deb1db1
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec59

In [5]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [6]:
import tensorflow as tf
import collections
import json
import os
import pandas as pd
import csv
from transformers import DistilBertTokenizer

max_seq_length = 64

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

REVIEW_BODY_COLUMN = "review_body"
REVIEW_ID_COLUMN = "review_id"

LABEL_COLUMN = "star_rating"
LABEL_VALUES = [1, 2, 3, 4, 5]

2024-01-15 09:26:25.725220: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 09:26:25.725284: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 09:26:25.725315: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 09:26:25.732970: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
label_map = {}
for (i, label) in enumerate(LABEL_VALUES):
    label_map[label] = i

In [8]:
print(label_map)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4}


In [9]:
class InputFeatures(object):
    """BERT特徴量ベクトル"""

    def __init__(self, input_ids, input_mask, segment_ids, label_id, review_id, date, label):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.review_id = review_id
        self.date = date
        self.label = label

In [10]:
class Input(object):
    """シーケンス分類で用いるトレーニング/テストの単一の入力"""

    def __init__(self, text, review_id, date, label=None):
        """入力のコンストラクタ
        Args:
          text: 文字列。トークン化されていない一つ目のシーケンスのテキスト。
            単一シーケンスのタスクではこのシーケンスのみを指定する。
          label: (オプショナル) 文字列。サンプルのラベル。トレーニングや検証用のサンプルでは指定する。
            テスト用のサンプルでは指定しない。
        """
        self.text = text
        self.review_id = review_id
        self.date = date
        self.label = label


In [11]:
def convert_input(the_input, max_seq_length):
    # まず、BERTが学習したデータ形式と合うようにデータを前処理する。
    # 1. テキストを小文字にする（BERT lowercaseモデルを用いる場合）
    # 2. トークン化する（例、"sally says hi" -> ["sally", "says", "hi"]）
    # 3. 単語をWordPieceに分割（例、"calling" -> ["call", "##ing"]）
    #
    # この辺りの処理はTransformersライブラリのトークナイザーがまかなってくれます。

    tokens = tokenizer.tokenize(the_input.text)
    tokens.insert(0, '[CLS]')
    tokens.append('[SEP]')
    # print("**{} tokens**\n{}\n".format(len(tokens), tokens))

    encode_plus_tokens = tokenizer.encode_plus(
        the_input.text,
        pad_to_max_length=True,
        max_length=max_seq_length,
        truncation=True
    )
    
    # 事前学習済みBERTの語彙ID。トークンを表す。（トークン数が `max_seq_length` 未満であれば0をパディングする）
    input_ids = encode_plus_tokens["input_ids"]

    # BERTがどのトークンに注目するかを0/1で指定。`input_ids` のパディング部分のベクトル要素には0を割り当てる。
    input_mask = encode_plus_tokens["attention_mask"]

    # テキスト分類のような単一シーケンスのタスクではセグメントIDは常に0とする。質問回答や次文予測のような2シーケンスタスクの場合は1を割り当てる。
    segment_ids = [0] * max_seq_length

    # それぞれのトレーニングデータの行のラベル（`star_rating` 1〜5）
    label_id = label_map[the_input.label]

    features = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_id=label_id,
        review_id=the_input.review_id,
        date=the_input.date,
        label=the_input.label,
    )

    # print("**{} input_ids**\n{}\n".format(len(features.input_ids), features.input_ids))
    # print("**{} input_mask**\n{}\n".format(len(features.input_mask), features.input_mask))
    # print("**{} segment_ids**\n{}\n".format(len(features.segment_ids), features.segment_ids))
    # print("**label_id**\n{}\n".format(features.label_id))
    # print("**review_id**\n{}\n".format(features.review_id))
    # print("**date**\n{}\n".format(features.date))
    # print("**label**\n{}\n".format(features.label))

    return features


In [12]:
def transform_inputs_to_tfrecord(inputs, output_file, max_seq_length):
    # データをBERTが理解できるフォーマットに変換する
    records = []
    tf_record_writer = tf.io.TFRecordWriter(output_file)

    for (input_idx, the_input) in enumerate(inputs):
        if input_idx % 10000 == 0:
            print("Writing input {} of {}\n".format(input_idx, len(inputs)))

        features = convert_input(the_input, max_seq_length)

        all_features = collections.OrderedDict()

        # input_ids、input_mask、segment_ids、label_idsを含んだTFRecordを作成
        all_features["input_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=features.input_ids))
        all_features["input_mask"] = tf.train.Feature(int64_list=tf.train.Int64List(value=features.input_mask))
        all_features["segment_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=features.segment_ids))
        all_features["label_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[features.label_id]))

        tf_record = tf.train.Example(features=tf.train.Features(feature=all_features))
        tf_record_writer.write(tf_record.SerializeToString())

        # Feature Storeに格納する、すべての特徴量を含んだレコードを作成
        records.append(
            {
                "input_ids": features.input_ids,
                "input_mask": features.input_mask,
                "segment_ids": features.segment_ids,
                "label_id": features.label_id,
                "review_id": the_input.review_id,
                "date": the_input.date,
                "label": features.label,
            }
        )

    tf_record_writer.close()

    return records

In [13]:
from datetime import datetime
from time import strftime

# timestamp = datetime.now().replace(microsecond=0).isoformat()
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(timestamp)

2024-01-15T09:26:42Z


In [14]:
import pandas as pd

data = [
    [
        5,
        "ABCD12345",
        """I needed an "antivirus" application and know the quality of Norton products.  This was a no brainer for me and I am glad it was so simple to get.""",
    ],
    [
        3,
        "EFGH12345",
        """The problem with ElephantDrive is that it requires the use of Java. Since Java is notorious for security problems I haveit removed from all of my computers. What files I do have stored are photos.""",
    ],
    [
        1,
        "IJKL2345",
        """Terrible, none of my codes worked, and I can't uninstall it.  I think this product IS malware and viruses""",
    ],
]

df = pd.DataFrame(data, columns=["star_rating", "review_id", "review_body"])

# Input クラスを使用して、データからサンプルを作成する。
inputs = df.apply(
    lambda x: Input(label=x[LABEL_COLUMN], text=x[REVIEW_BODY_COLUMN], review_id=x[REVIEW_ID_COLUMN], date=timestamp),
    axis=1,
)

In [15]:
df

,star_rating,review_id,review_body
0,5,ABCD12345,"I needed an ""antivirus"" application and know t..."
1,3,EFGH12345,The problem with ElephantDrive is that it requ...
2,1,IJKL2345,"Terrible, none of my codes worked, and I can't..."


In [16]:
inputs

0    <__main__.Input object at 0x7f0dd9b01960>
1    <__main__.Input object at 0x7f0dd9b01a80>
2    <__main__.Input object at 0x7f0dd9b016c0>
dtype: object

In [17]:
# date が Feature Store の仕様に合わせて ISO-8601 になっていることを確認
print(inputs[0].date)

2024-01-15T09:26:42Z


In [18]:
output_file = "./data.tfrecord"

In [19]:
records = transform_inputs_to_tfrecord(inputs, output_file, max_seq_length)

Writing input 0 of 3



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [20]:
print(records[0]['input_ids'])

[101, 1045, 2734, 2019, 1000, 3424, 23350, 1000, 4646, 1998, 2113, 1996, 3737, 1997, 10770, 3688, 1012, 2023, 2001, 1037, 2053, 4167, 2121, 2005, 2033, 1998, 1045, 2572, 5580, 2009, 2001, 2061, 3722, 2000, 2131, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
import pandas as pd

In [21]:
df = pd.read_parquet("/mnt/amazon_reviews_2015_small.snappy.parquet",columns=["star_rating","review_id","review_body"])

In [22]:
for i in range(1,6):
    print("star_rating: ", i)
    print(df[df['star_rating'] == i].count())
    print('-------------------------')

star_rating:  1
star_rating    36844
review_id      36844
review_body    36844
dtype: int64
-------------------------
star_rating:  2
star_rating    22482
review_id      22482
review_body    22482
dtype: int64
-------------------------
star_rating:  3
star_rating    37827
review_id      37827
review_body    37827
dtype: int64
-------------------------
star_rating:  4
star_rating    77271
review_id      77271
review_body    77271
dtype: int64
-------------------------
star_rating:  5
star_rating    325576
review_id      325576
review_body    325576
dtype: int64
-------------------------


In [23]:
for i in range(1,6):
    new_df = 'df' + str(i)
    globals()[new_df] = df[df['star_rating'] == i].sample(n=22482)

In [24]:
df1

,star_rating,review_id,review_body
469830,1,R2N50EVZBQ82B0,Too small and fits weird not flattering at all...
208163,1,R1372MRPSX29ET,Wasn't good
101708,1,RBZK0V7QCM3EJ,Runs too small. I ordered a large for my Boxer...
424283,1,R3TD70NF071DOW,It is NOTHING like the picture showed. I EXPEC...
244037,1,R39DSQK0FPD4ZA,This gave me allergies :(
...,...,...,...
34192,1,R348TX7UIKY9O6,We did not receive the correct poster. My son ...
497268,1,RPLLNRIQ96JGJ,Very dangerous !!!! They got so hot on my cli...
422215,1,R16PSWBUDMSQM1,Buyers beware! This product did not contain ca...
481150,1,R1VNKSGFFJND2I,"So, not being a musician and just trying to ge..."


In [25]:
df2

,star_rating,review_id,review_body
227551,2,R6T1JUMAZJZ8U,I ordered several pictures of Marilyn Monroe. ...
165992,2,R1QVZ11HMK4HLA,I wish I had bought another one and velcrowed ...
173146,2,R26Z3JHIAHZBH3,I gave it two stars because I was able to fini...
90470,2,RLP69BJ93751X,These are not Gripstix and I felt they did not...
357659,2,R30OY4QU6BR40A,the listing said is white but it come peach pi...
...,...,...,...
107423,2,R3MDAKOPLISTVR,did not like the product it had a chalky taste...
288731,2,R1VFVYGPGWH7PT,"Cheap material, great for a Halloween costume."
14959,2,R35C8TYMDG1NPA,Bought this for my mother in-laws house which ...
458646,2,RMVIY683NVQGB,It was okay. The story was very drawn out and ...


In [26]:
df3

,star_rating,review_id,review_body
265403,3,R1NF492BOUS62L,Picture was not crisp on the shirt. Poor quali...
130395,3,R16G3YYWZ3B9RT,"This movie was pretty stupid, right? Well, it ..."
180536,3,RA08OJ37VP3P9,I only really cared for one song
226149,3,R2OOZOAZO65HT3,Be careful to measure properly for this item. ...
239154,3,R17B37J3EKHQZN,I bought this stroller because a toddler and a...
...,...,...,...
445658,3,R39FI0CHJNF1N6,I ordered a small then asked for a medium they...
116238,3,R38FGHC45ILYVV,Good picture is the best I can say about this ...
492086,3,RS894B72X4GVX,Essentially just applesauce with other flavors...
80739,3,R2F2T70HGK3IWR,Cup is a little small but shipped within a wee...


In [27]:
df4

,star_rating,review_id,review_body
137861,4,RQ6JQN5K0LDA6,"I was quite impressed with this game, which is..."
158825,4,RSFL9G7I3LAOA,A reasonable substitute for gold leaf. It is ...
424226,4,R237SGCE8BEGGU,"So, comfortable. Love them."
69033,4,R1H4RRKDT0AC93,Gave it a 4 star because it still had to much ...
136188,4,R2JRPQAHCIQVOR,i have enjoyed playing this game. i didnt give...
...,...,...,...
246474,4,RF6GYWR5P6EJU,"I've had this for @5 months now and Toby, my 1..."
125784,4,R2M981BTXXZ9TL,This book encourages the reader to believe she...
239225,4,R11FB74F1WS67Q,We purchased the ottoman glider to go with the...
347354,4,R3J2Y4OC7TEA2A,Great sound...good quality.


In [28]:
df5

,star_rating,review_id,review_body
340558,5,R15XFI81K6C5HA,I'm happy with my purchase the wig was exactly...
65728,5,R1B56HISMEPDCE,I had been using a Sigma 18-250mm lens as a wa...
93178,5,REOAP9J8Q6UF6,Best thing ever for making sallad. Staffan
156695,5,R3KRZ6CGL415RV,Super comfortable!
459620,5,R2WJYHB1OJ0498,Great story as all of yours are. They are bes...
...,...,...,...
436505,5,RXBCN0ADOZFIH,great product.
80993,5,R39KQ4UEM7REZ0,It's perfect. It's well made. And exactly what...
237543,5,R2G502Y62EPEW4,Lots of fun to make and all my workmates have ...
179510,5,R1HYWHMABJX1HF,loveed....


In [29]:
df_temp = pd.concat([df1,df2,df3,df4,df5])

In [30]:
df_temp.count()

star_rating    112410
review_id      112410
review_body    112410
dtype: int64

In [31]:
df_shaffle = df_temp.sample(100000)

In [32]:
df_head = df_shaffle.head(80000)
df_tail = df_shaffle.tail(20000)

In [33]:
df_head.to_parquet("/mnt/amazon_reviews_2015_small_head.snappy.parquet")
df_tail.to_parquet("/mnt/amazon_reviews_2015_small_tail.snappy.parquet")

In [45]:
train_df = df_head.sample(n=80000)
train_df

,star_rating,review_id,review_body
281907,1,ROH3DDWBARFXN,"Don't buy, go hard wire."
177049,1,R1C3TFQ3ADRXVK,Terrible quality. Sending these back. We bough...
426701,1,R1ZFP81ABZS8VQ,Not for my Very thick Very curly hair!
252421,2,RT68PXU8WXDEA,"Did not fit my daughter. She liked the design,..."
212122,4,RF4V44C27PMMW,I've only had this for a couple of days but so...
...,...,...,...
198778,5,R1R1IY9TYMUEP6,"I started with 35mm many years ago, and made a..."
41828,3,R5OE879KLMQSL,The price is exactly right. But the features a...
499297,1,R1VA0NWPFJF3G,This gizmo is all plastic. The place where it ...
340956,4,R3SO6DY5LZE7L2,I generally like the Olay products and this on...


In [46]:
validation_df = df_tail.sample(n=20000)
validation_df

,star_rating,review_id,review_body
483850,5,R5R2L29XK3J2W,This is a total life saver! I was about ready...
448516,2,R2A5Q17WV771A5,"Get a bigger size that your usual, got a small..."
225318,2,R198LNCNL0VFCA,These don't last the night like Breathe Right ...
250618,1,R3TEWST14UGTS0,Didn't like them at all. I ordered this bootie...
279638,1,R1W7VN593A9UW,Good
...,...,...,...
288774,2,R1TY7NVTLLUGJJ,It is cute but somewhat opaque (you can see yo...
315281,1,R1SQHHLWRMQMPD,This is a stupid game
378662,4,R3VCNWMXZ5S7GY,Midnight is the typical southern small town. T...
323005,1,R3NJK69DSNS8E0,I couldn't get it to grow.


In [47]:
print(f"df Memory Usage: {df.memory_usage(deep=True).sum() / 1024**3} GB")

df Memory Usage: 0.17162852175533772 GB


In [48]:
# Input クラスを使用して、データからサンプルを作成する。
train_inputs = train_df.apply(
    lambda x: Input(label=x[LABEL_COLUMN], text=x[REVIEW_BODY_COLUMN], review_id=x[REVIEW_ID_COLUMN], date=timestamp),
    axis=1,
)

In [49]:
# Input クラスを使用して、データからサンプルを作成する。
validation_inputs = validation_df.apply(
    lambda x: Input(label=x[LABEL_COLUMN], text=x[REVIEW_BODY_COLUMN], review_id=x[REVIEW_ID_COLUMN], date=timestamp),
    axis=1,
)

In [ ]:
# !pip install pandarallel

In [ ]:
# from pandarallel import pandarallel
# import os
# os.environ['JOBLIB_TEMP_FOLDER'] = '/tmp'
# !pip3 install --no-cache-dir accelerate
# pandarallel.initialize(nb_workers=2, progress_bar=True, use_memory_fs=False)

In [ ]:
#def func(x):
#    return lambda x: Input(label=x[LABEL_COLUMN], text=x[REVIEW_BODY_COLUMN], review_id=x[REVIEW_ID_COLUMN], date=timestamp)

# inputs = df.parallel_apply(
#     lambda x: Input(label=x[LABEL_COLUMN], text=x[REVIEW_BODY_COLUMN], review_id=x[REVIEW_ID_COLUMN], date=timestamp),
#     axis=1
# )

In [50]:
train_inputs

281907    <__main__.Input object at 0x7f0dd9bc6da0>
177049    <__main__.Input object at 0x7f0dd9bc7190>
426701    <__main__.Input object at 0x7f0dad4334f0>
252421    <__main__.Input object at 0x7f0dad424e20>
212122    <__main__.Input object at 0x7f0dad424dc0>
                            ...                    
198778    <__main__.Input object at 0x7f0dac679570>
41828     <__main__.Input object at 0x7f0dac6795d0>
499297    <__main__.Input object at 0x7f0dac679630>
340956    <__main__.Input object at 0x7f0dac679690>
328466    <__main__.Input object at 0x7f0dac6796f0>
Length: 80000, dtype: object

In [51]:
train_output_file = "/mnt/train_small_data.tfrecord"

In [52]:
validation_output_file = "/mnt/validation_small_data.tfrecord"

In [51]:
# date が Feature Store の仕様に合わせて ISO-8601 になっていることを確認
# print(train_inputs[0].date)

In [53]:
train_records = transform_inputs_to_tfrecord(train_inputs, train_output_file, max_seq_length)

Writing input 0 of 80000



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Writing input 10000 of 80000

Writing input 20000 of 80000

Writing input 30000 of 80000

Writing input 40000 of 80000

Writing input 50000 of 80000

Writing input 60000 of 80000

Writing input 70000 of 80000



In [54]:
validation_records = transform_inputs_to_tfrecord(validation_inputs, validation_output_file, max_seq_length)

Writing input 0 of 20000

Writing input 10000 of 20000



In [54]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>